# Physics 5300 Final Problem Set - Animating Double Pendulum

## aka Pendulum Object Oriented Programming (P.O.O.P.)

Created 04-22-19 by Lucas Nestor  
Revised 04-24-19 by Lucas Nestor

In the last notebook we analyzed a double pendulum and saw how it could create chaotic motion. In this notebook we will animate the pendulum to help visualize it's motion.

In [ ]:
import numpy as np
from numpy import linalg as LA
import matplotlib.pyplot as plt
from scipy.integrate import solve_ivp

from matplotlib import animation
from IPython.display import HTML

In [ ]:
%matplotlib inline

In [ ]:
class DoublePendulum():
    """
    Implements Langrange's equations for a double pendulum.
    """
    
    def __init__(self, m1=1., m2=1., L1=1., L2=1., g=1.):
        self.m1 = m1
        self.m2 = m2
        self.L1 = L1
        self.L2 = L2
        self.g = g
        
    def M_matrix(self, y):
        """
        Returns the matrix multiplying the second time derivative of theta.
        
        Parameters
        ==========
        y : float
            A vector with y[0] = theta_1, y[1] = theta_2, y[2] = dtheta_1/dt, y[3] = dtheta_2/dt
        """
        theta_1 = y[0]
        theta_2 = y[1]
        cosine = np.cos(theta_1 - theta_2)
        
        return np.array([
            [(self.m1 + self.m2) * self.L1**2    , self.m2 * self.L1 * self.L2 * cosine],
            [self.m2 * self.L1 * self.L2 * cosine, self.m2 * self.L2**2             ]
        ])
    
    def h_vec(self, y):
        """
        Returns the RHS for the matrix form of Lagrange's equationa
        
        Parameters
        ==========
        y : float
            A vector with y[0] = theta_1, y[1] = theta_2, y[2] = dtheta_1/dt, y[3] = dtheta_2/dt
        """
        theta_1 = y[0]
        theta_2 = y[1]
        theta_1_dot = y[2]
        theta_2_dot = y[3]
        sine = np.sin(theta_1 - theta_2)
        
        return np.array([
            [(self.m1 + self.m2) * self.g * self.L1 * np.sin(theta_1)
             + self.m2 * self.L1 * self.L2 * theta_2_dot**2 * sine],
            [self.m2 * self.g * self.L2 * np.sin(theta_2)
             - self.m2 * self.L1 * self.L2 * theta_1_dot**2 * sine]
        ])
    
    def dy_dt(self, t, y):
        """
        This function returns the right-hand side of the diffeq: 
        [dtheta_vec/dt d^2theta_vec/dt^2]
        
        Parameters
        ----------
        t : float
            time 
        y : float
            vector with y[0] = theta_1, y[1] = theta_2, y[2] = dtheta_1/dt, y[3] = dtheta_2/dt
        """
        
        dy_dt_vec = np.zeros(4)
        dy_dt_vec[0:2] = y[2:4]
        dy_dt_vec[2:4] = (-1 * LA.inv(self.M_matrix(y)) @ self.h_vec(y)).T

        return dy_dt_vec
    
    def solve_ode(self, t_pts, initial_conditions, abserr=1.0e-10, relerr=1.0e-10):
        """
        Solves the motion for a given set of initial conditions.
        
        Parameters
        ==========
        t_pts : float
                time
        initial_conditions : float
                             array in the form of [theta1(0), theta2(0), theta1_dot(0), theta2_dot(0)]
        abserr : float
                 absolute error threshold
        relerr : float
                relative error threshold
        """
        return solve_ivp(self.dy_dt, (t_pts[0], t_pts[-1]), initial_conditions, 
                         t_eval=t_pts, rtol=relerr, atol=abserr).y

## Pendulum Animation Class

Next we will make a class to help us with animating our pendulums. This will allow us to show chaotic motion by superimposing pendulums on top of each other easily.

In [ ]:
def cartesian_coords(L1, L2, theta_1, theta_2):
    """
    Converts polar to Cartesian coordinates.
    
    Parameters
    ==========
    L1 : float
         length of first pendulum
    L2 : float
         length of second pendulum
    theta_1 : float
              angle of first pendulum
    theta_2 : float
              angle of second pendulum
    """
    x1 = L1 * np.sin(theta_1)
    y1 = - L1 * np.cos(theta_1)
    x2 = x1 + L2 * np.sin(theta_2)
    y2 = y1 - L2 * np.cos(theta_2)
        
    return (x1, y1, x2, y2)


Here are some small helper classes that break out functionality of animating the pendulum.

In [ ]:
class PendulumTrail():
    """
    Adds a small line behind a pendulum mass to show it's recent trajectory.
    """
    def __init__(self, pos=(0,0), length=10, color='k', ax=None):
        """
        Parameters
        ==========
        pos : float
              inital position in form (x,y)
        length : int
                 Number of points to have on the trail. For example,
                 if length = 10, the 10th oldest point will be
                 replaced next
        color : color
                the color of the trail
        ax : axis
             the axis on which the trail will be plotted
        """
        x0, y0 = pos
        self.trail, = ax.plot(np.ones(length) * x0, np.ones(length) * y0, color=color, alpha=0.4, zorder=1)
    
    def add_value(self, pos=(0,0)):
        """
        Adds a new value on to the trail, which deletes the current oldest value.
        
        Parameters
        ==========
        pos : float
              new value to add in form (x, y)
        """
        x, y = pos
        current_x, current_y = self.trail.get_data()
        
        current_x[-1] = x
        current_y[-1] = y
        
        self.trail.set_data(np.roll(current_x, 1), np.roll(current_y, 1))
        
class PendulumShape():
    """
    Handles updating the data for a pendulum rod, mass, and trail
    """
    def __init__(self, start_pos=(0,0), end_pos=(1,1), mass_trail=True, trail_length=10, ax=None):
        """
        Parameters
        ==========
        start_pos : float
                    the position where the rod of the pendulum starts
        end_pos : float
                  the position where the mass of the pendulum sits
        mass_trail : bool
                     true if the trail behind the pendulum mass is to be animated
        trail_length : integer
                       the length of the trail of the pendulum's path
        ax : axis
             the pyplot axis on which the pendulum will be plotted
        """
        self.mass_trail = mass_trail
        
        x0, y0 = start_pos
        x1, y1 = end_pos
        
        self.mass, = ax.plot(x1, y1, 'o', markersize=10, zorder=10)
        self.rod, = ax.plot([x0, x1], [y0, y1], color='black', lw=3, zorder=5)
        if self.mass_trail: self.trail = PendulumTrail(pos=end_pos, color=self.color(), length=trail_length, ax=ax)

    def update(self, start_pos=(0,0), end_pos=(1,1)):
        """
        Updates the position of the pendulum.
        
        Parameters
        ==========
        start_pos : float
                    the position where the rod of the pendulum starts
        end_pos : float
                  the position where the mass of the pendulum sits
        """
        x0, y0 = start_pos
        x1, y1 = end_pos
        
        if self.mass_trail: self.trail.add_value(pos=(x1,y1))
        self.rod.set_data([x0, x1], [y0, y1])
        self.mass.set_data(x1, y1)
    
    def color(self):
        return self.mass.get_color()
    
class PendulumGraph():
    """
    Adds a plot of theta vs time for a graph and animates a marker
    at the current position of the pendulum.
    """
    def __init__(self, t_pts, theta, color='blue', ax=None):
        """
        Parameters
        ==========
        t_pts : float
                time
        theta : float
                angle of the pendulum
        color : string
                the color of the plot
        ax : axis
             the pyplot axis on which the graph will be plotted
        """
        self.t_pts = t_pts
        self.theta = theta
        ax.plot(t_pts, theta, color=color, zorder=1)
        self.marker, = ax.plot(t_pts[0], theta[0], 'o', markersize=6, color=color, zorder=10)
        
    def update(self, index):
        self.marker.set_data(self.t_pts[index], self.theta[index])


In [ ]:
class PendulumAnimation():
    """
    Animates a double pendulum
    """
    def __init__(self, t_pts=None, mass_trail=True):
        """
        Parameters
        ==========
        t_pts : float
                time to evaluate the pendulum's motion
        mass_trail : bool
                       true if the trail behind each pendulum is to be animated
        """
        self.t_pts = t_pts
        self.mass_trail = mass_trail
        self.pendulums = []
        self.plotted_data = []
        
    def add_pendulum(self, pendulum, initial_conditions):
        """
        Adds a pendulum to the list to be animated
        
        Parameters
        ==========
        pendulum : DoublePendulum object
                   pendulum object to be evaluated
        initial_conditions : float
                             list in the form 
                                 [theta_1(0), theta_2(0), theta_1_dot(0), theta_2_dot(0)]
        """
        theta_1, theta_2, _, _ = pendulum.solve_ode(self.t_pts, initial_conditions)
        self.pendulums.append((theta_1, theta_2, pendulum.L1, pendulum.L2))
        
    def setup_plot(self, title=None):
        """
        Creates the figure to be ready for animating
        
        Parameters
        ==========
        title : string
                title to give to the plot
        """
        self.fig = plt.figure(figsize=(15,5))
        ax_physical = self.fig.add_subplot(1,2,1)
        ax_physical.plot(0, 0, 'o', color='black', markersize=6)
        
        ax_motion = self.fig.add_subplot(1,2,2)
        
        for (theta_1, theta_2, L1, L2) in self.pendulums:            
            x1, y1, x2, y2 = cartesian_coords(L1, L2, theta_1[0], theta_2[0])
            
            pend1 = PendulumShape(start_pos=(0,0), end_pos=(x1,y1), mass_trail=self.mass_trail,
                                  trail_length=10, ax=ax_physical)
            pend2 = PendulumShape(start_pos=(x1,y1), end_pos=(x2,y2), mass_trail=self.mass_trail,
                                  trail_length=10, ax=ax_physical)
            
            marker1 = PendulumGraph(self.t_pts, theta_1, color=pend1.color(), ax=ax_motion)
            marker2 = PendulumGraph(self.t_pts, theta_2, color=pend2.color(), ax=ax_motion)
            
            self.plotted_data.append((pend1, pend2, marker1, marker2))
                
        max_len = self.pendulums[0][2] + self.pendulums[0][3] # assuming all pendulums same length
        gap = 0.1 * max_len
        ax_physical.set_xlim(-max_len - gap, max_len + gap)
        ax_physical.set_ylim(-max_len - gap, max_len + gap)
        ax_physical.set_aspect('equal')
        
        ax_motion.set_xlim(self.t_pts[0], self.t_pts[-1])
        
        if title: ax.set_title(title)
    
    def next_animation_frame(self, i):
        """
        Returns the next fram of an animation
        
        Parameters
        ==========
        i : integer
            frame index
        """
        skip_index = i * self.skip
        for index, (pend1, pend2, marker1, marker2) in enumerate(self.plotted_data):
            theta_1, theta_2, L1, L2 = self.pendulums[index]

            x1, y1, x2, y2 = cartesian_coords(L1, L2, theta_1[skip_index], theta_2[skip_index])
              
            pend1.update(start_pos=(0,0), end_pos=(x1,y1))
            pend2.update(start_pos=(x1,y1), end_pos=(x2,y2))
            
            marker1.update(skip_index)
            marker2.update(skip_index)

        
    def create_animation(self, frame_interval=10., num_frames=None, skip=1):
        """
        Creates an animation object for the pendulums added before called.
        
        Parameters
        ==========
        frame_interval : float
                         time between frames
        num_frames : integer
                     number of frames to animate
        skip : integer
               number of array points to skip per frame
        """
        
        self.skip = skip
        if not num_frames: num_frames = int(self.t_pts[-1] / (self.t_pts[1] - self.t_pts[0]) / self.skip)
            
        self.anim = animation.FuncAnimation(self.fig, 
                               self.next_animation_frame, 
                               init_func=None,
                               frames=num_frames, 
                               interval=frame_interval, 
                               blit=False,
                               repeat=False)
        return self.anim


## Various Animations

Now that we have our classes defined, we can look at different animations. First we'll look at a single pendulum with some interesting initial conditions.

In [ ]:
%%capture
pend_anim = PendulumAnimation(t_pts=np.arange(0., 40., 0.1), mass_trail=True)

theta_1_0 = 2 * np.pi / 3
theta_2_0 = np.pi / 3
theta_1_dot_0 = 0 
theta_2_dot_0 = 0
initial_conditions = [theta_1_0, theta_2_0, theta_1_dot_0, theta_2_dot_0]

L1 = 1.
L2 = 1.
pend = DoublePendulum(L1=L1, L2=L2)

pend_anim.add_pendulum(pend, initial_conditions)

pend_anim.setup_plot()
anim = pend_anim.create_animation(frame_interval=50, skip=1)


In [ ]:
HTML(anim.to_jshtml())

Next we'll look at two pendulums and visualize their sensitivity to initial conditions.

In [ ]:
%%capture
pend_anim = PendulumAnimation(t_pts=np.arange(0., 100., 0.2), mass_trail=False)

theta_1_0 = np.pi / 2
theta_2_0 = np.pi / 3
theta_1_dot_0 = 0 
theta_2_dot_0 = 2
initial_conditions = [theta_1_0, theta_2_0, theta_1_dot_0, theta_2_dot_0]
initial_conditions_2 = [theta_1_0, theta_2_0 + 0.001, theta_1_dot_0, theta_2_dot_0]

L1 = 2.
L2 = 1.
pend = DoublePendulum(L1=L1, L2=L2)

pend_anim.add_pendulum(pend, initial_conditions)
pend_anim.add_pendulum(pend, initial_conditions_2)

pend_anim.setup_plot()
anim = pend_anim.create_animation(frame_interval=50, skip=2)


In [ ]:
HTML(anim.to_jshtml())

Now we'll just be dumb.

In [ ]:
%%capture
pend_anim = PendulumAnimation(t_pts=np.arange(0., 50., 0.2), mass_trail=False)

theta_1_0 = 0
theta_2_0 = 0
theta_1_dot_0 = 0
theta_2_dot_0 = 5
initial_conditions = [theta_1_0, theta_2_0, theta_1_dot_0, theta_2_dot_0]

m1=20.
m2=1.
L1 = 3.
L2 = 1.
pend = DoublePendulum(m1=m1, m2=m2, L1=L1, L2=L2)

pend_anim.add_pendulum(pend, initial_conditions)

pend_anim.setup_plot()
anim = pend_anim.create_animation(frame_interval=50)


In [ ]:
HTML(anim.to_jshtml())